If you have a word file than a pdf , here is the code to convert word to pdf

In [ ]:
from docx import Document
from fpdf import FPDF

# Function to convert Word to PDF
def convert_word_to_pdf(word_file_path, pdf_file_path):

    doc = Document(word_file_path)

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    for paragraph in doc.paragraphs:
        pdf.multi_cell(0, 10, paragraph.text)

    pdf.output(pdf_file_path)
    print(f"PDF saved to {pdf_file_path}")

word_file_path = '/content/your_word_file.docx'
pdf_file_path = '/content/your_output_file.pdf'

convert_word_to_pdf(word_file_path, pdf_file_path)


In [ ]:
from google.colab import files
uploaded = files.upload()
from google.colab import files
files.download(pdf_file_path)

imports and external downloads

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher  # For fuzzy matching

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading of data from excel(company details )
and resume(user side)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
excel_file = '/content/drive/MyDrive/internship_data.xlsx'
df = pd.read_excel(excel_file)
print("Column Names:", df.columns)
skills_column = df['Skills_Required']
print(skills_column)

Column Names: Index(['Sno', 'Profile', 'Company', 'Location', 'Start Date', 'Duration',
       'Stipend', 'Unnamed: 7', 'Apply By', 'About Internship',
       'Skills_Required', 'Who Can Apply', 'About Company'],
      dtype='object')
0       ['Business Analysis', 'Business Research', 'Ef...
1       ['Advanced Excel', 'Facebook Ads', 'Google Ana...
2        ['Effective Communication', 'MS-Excel', 'Sales']
3       ['Certificate', 'Letter of recommendation', 'F...
4       ['Adobe XD', 'Figma', 'PROTOTYPING', 'Sketch',...
                              ...                        
1629    ['Adobe After Effects', 'Adobe Creative Suite'...
1630    ['Creative Writing', 'Digital Marketing', 'Fac...
1631    ['Effective Communication', 'English Proficien...
1632    ['Effective Communication', 'English Proficien...
1633    ['Effective Communication', 'English Proficien...
Name: Skills_Required, Length: 1634, dtype: object


In [ ]:
excel_file = '/content/drive/MyDrive/internship_data.xlsx'
df = pd.read_excel(excel_file)

Extraction and cleaning of data using pdfplumber

extraction of data is done by checking "Skills" section in the resume then next text after the word""skills" is taken as skills , garbage of the skills extration is done by if words is not seperated by comma or continous text is used wioth out gap is recongised as garbage , since all of the resume skills section either has a comma seperated value or set space seperated value in it .

In [ ]:

skills_column = df['Skills_Required']
unique_skills = set()

for skills in skills_column:
    skill_list = str(skills).split(',')
    for skill in skill_list:
        clean_skill = re.sub(r'[{}[\]"\']', '', skill).strip().lower()
        if len(clean_skill) > 1 and not clean_skill.isdigit():
            unique_skills.add(clean_skill)

final_skill_list = sorted(unique_skills)


In [ ]:

def extract_skills(text):
    raw_skills = text.split(',')
    cleaned_skills = [skill.strip().lower() for skill in raw_skills if len(skill.strip().split()) <= 2]
    return cleaned_skills


In [ ]:
# Example skills extracted from resume (replace with actual extraction logic)
import pdfplumber
with pdfplumber.open('/content/drive/MyDrive/resume/resume/10089434.pdf') as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        skills_index = text.find('Skills')

        if skills_index != -1:
            skills_section = text[skills_index:skills_index + int(len(text) * 0.25)]
            break
skills_list = extract_skills(skills_section)

excel_skills = [skill.lower().strip() for skill in final_skill_list]
resume_skills = [skill.lower().strip() for skill in skills_list]

Skill matching with the "fuzzy matching" logic .
fuzzy logic is an approach to variable processing that allows for multiple possibl truth values to be processed throgh the same varibale.

In [ ]:
# Fuzzy matching function (for skill matching)
def fuzzy_match(skill, skills_required):

    return SequenceMatcher(None, skill, skills_required).ratio()

def convert_stipend(stipend):
    stipend = stipend.replace('₹', '').strip().lower()

    if 'unpaid' in stipend:
        return 0.0

    stipend = stipend.replace(',', '')

    if '/week' in stipend:
        weekly_value = stipend.replace('/week', '').strip()
        try:
            weekly_value = float(re.sub(r'[^\d.]', '', weekly_value))
            return weekly_value * 4
        except ValueError:
            return 0.0

    stipend = stipend.replace('/month', '').strip()
    if '-' in stipend:
        try:
            low, high = stipend.split('-')
            return (float(low.strip()) + float(high.strip())) / 2
        except ValueError:
            return 0.0

    try:
        return float(stipend)
    except ValueError:
        return 0.0

Mapping the taken set of skills with the resume and mapping eith the company skills using fuzzy matching logic

In [ ]:
# Matching logic: Map resume skills to the companies that require them using fuzzy matching
company_count = {}
for skill in resume_skills:
    for index, row in df.iterrows():
        if isinstance(row['Skills_Required'], str):
            skills_required = row['Skills_Required'].lower()
            if any(fuzzy_match(skill, req_skill) > 0.7 for req_skill in skills_required.split(',')):  # Fuzzy matching
                company = row['Company']
                if company not in company_count:
                    company_count[company] = {'matches': 0, 'stipend': 0.0}
                company_count[company]['matches'] += 1

excess information and sorting with not only the highest matched skills but also with the amount of stipend given to the intern joinee by method of hybrid (90 percent ) and normalized (10 percent)

In [ ]:
# Add stipend information and normalize for each company
max_stipend = 0
for company in company_count:
    stipend_value = df[df['Company'] == company]['Stipend'].values[0]
    stipend_value = convert_stipend(stipend_value)
    company_count[company]['stipend'] = stipend_value
    max_stipend = max(max_stipend, stipend_value)

In [ ]:
# Hybrid scoring: 90% skill match, 10% normalized stipend
recommendations = []
for company, values in company_count.items():
    skill_match_score = values['matches'] * 0.9
    stipend_score = (values['stipend'] / max_stipend) * 0.1 if max_stipend > 0 else 0
    total_score = skill_match_score + stipend_score

    company_details = df[df['Company'] == company].iloc[0]
    recommendations.append({
        'Company': company,
        'Profile': company_details['Profile'],
        'Location': company_details['Location'],
        'Stipend': company_details['Stipend'],
        'Matched Skills': values['matches'],
        'Total Score': total_score
    })

Sorting of the filtered comapny list in order of hybrid (90 % of skill matching )and standard (10% stipend matching)  

In [ ]:

recommendations = sorted(recommendations, key=lambda x: x['Total Score'], reverse=True)

# Display the top 10 recommended internship companies
print("\nRecommended Internship Companies:")
for i, rec in enumerate(recommendations[:10], 1):
    print(f"Company {i}: {rec['Company']}, Profile: {rec['Profile']}, Location: {rec['Location']}, "
          f"Stipend: {rec['Stipend']}, Matched Skills: {rec['Matched Skills']}, Total Score: {rec['Total Score']:.2f}")


Recommended Internship Companies:
Company 1: Pawzz Foundation, Profile: Youth Empowerment, Location: Work from home, Stipend: ₹ 1,500-10,000 /month, Matched Skills: 3, Total Score: 2.73
Company 2: Samridh Harshit, Profile: Full Stack Development, Location: Work from home, Stipend: ₹ 1,500 /month +  Incentives, Matched Skills: 2, Total Score: 1.81
Company 3: The Super Focus Club, Profile: Software Development, Location: Work from home, Stipend: ₹ 15,000-25,000 /month, Matched Skills: 1, Total Score: 1.00
Company 4: AIMonk Labs Technology Limited, Profile: Deep Learning, Location: Work from home, Stipend: ₹ 15,000-25,000 /month, Matched Skills: 1, Total Score: 1.00
Company 5: Bilateral Solutions Private Limited, Profile: Digital Marketing (Work From Office), Location: Mandi, Stipend: ₹ 20,000 /month, Matched Skills: 1, Total Score: 1.00
Company 6: Dalisoft Technologies Private Limited, Profile: Software Development, Location: Gurgaon, Stipend: ₹ 15,000-25,000 /month, Matched Skills: 1, 

Storing of the final list to the excel sheet

In [ ]:
# Save recommendations to Excel
recommended_df = pd.DataFrame(recommendations)
recommended_df.to_excel('/content/drive/MyDrive/recommended_internships_fuzzy_hybrid.xlsx', index=False)

**ML model approach using vectorization **

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

excel_file = '/content/drive/MyDrive/internship_data.xlsx'
df = pd.read_excel(excel_file)

Extract skills required by companies and clean up

In [ ]:

df['Skills Required'] = df['Skills_Required'].str.lower().str.replace(r'[{}[\]"\']', '', regex=True)

skills_section = resume_skills

Create a TF-IDF Vectorizer
Fit the model on the 'Skills Required' and transform into TF-IDF matrix
generalize the skills
Calculate cosine similarity between the resume and the internship skills
add similarity score to dataframe

In [ ]:

tfidf_vectorizer = TfidfVectorizer()


tfidf_matrix = tfidf_vectorizer.fit_transform(df['Skills Required'])

# Transform the resume skills into TF-IDF

resume_tfidf = tfidf_vectorizer.transform([' '.join(skills_section).lower()])

cosine_similarities = cosine_similarity(resume_tfidf, tfidf_matrix)

df['Similarity Score'] = cosine_similarities.flatten()

Sort the DataFrame based on similarity score and get the top 10 recommendations


In [ ]:

recommended_internships = df.sort_values(by='Similarity Score', ascending=False).head(10)

print("\nRecommended Internship Companies:")
for index, row in recommended_internships.iterrows():
    print(f"Company: {row['Company']}, Profile: {row['Profile']}, Location: {row['Location']}, "
          f"Stipend: {row['Stipend']}, Similarity Score: {row['Similarity Score']:.4f}")


Recommended Internship Companies:
Company: Requin Solutions Private Limited, Profile: Subject Matter Expert (Civil Engineering), Location: Jaipur, Stipend: ₹ 10,000 /month, Similarity Score: 0.3679
Company: Hamari Pahchan NGO, Profile: General Management, Location: Delhi, South, Stipend: ₹ 3,000-5,000 /month, Similarity Score: 0.3667
Company: Dalisoft Technologies Private Limited, Profile: Software Development, Location: Gurgaon, Stipend: ₹ 15,000-25,000 /month, Similarity Score: 0.3653
Company: Doodhvale, Profile: Business Strategy, Location: Delhi, Stipend: ₹ 7,000 /month, Similarity Score: 0.3186
Company: Robokart, Profile: Electrical Engineering, Location: Ahmedabad, Bhavnagar, Gandhinagar, Mehsana, Surat, Anand, Navsari, Vadodara, Dahod, Bharuch, Sabarkatha, Panchmahal, Banaskantha, Mahisagar, Narmada, Stipend: ₹ 5,000 /month, Similarity Score: 0.3139
Company: Senso Vision System, Profile: SAS Marketing Intern, Location: Bangalore, Stipend: ₹ 8,000 /month, Similarity Score: 0.261

Optionally, save recommendations to a new Excel file

In [ ]:
recommended_internships.to_excel('/content/drive/MyDrive/recommended_internships_ml.xlsx', index=False)

In [ ]:

recommended_internship = recommended_internships

similarity_threshold = 0.2
recommended_internships['Similarity Score'] = pd.to_numeric(recommended_internships['Similarity Score'])
relevant_retrieved = sum(1 for _, internship in recommended_internships.iterrows() if internship['Similarity Score'] > similarity_threshold)
total_retrieved = len(recommended_internships)

total_actual_relevant = relevant_retrieved


precision = relevant_retrieved / total_retrieved if total_retrieved != 0 else 0
recall = relevant_retrieved / total_actual_relevant if total_actual_relevant != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f'Estimated Precision: {precision:.2f}')
print(f'Estimated Recall: {recall:.2f}')
print(f'Estimated F1 Score: {f1_score:.2f}')


Estimated Precision: 0.80
Estimated Recall: 1.00
Estimated F1 Score: 0.89
